# Finetuning an LLM
An adventure by Fred

### Requirements

In [1]:
! pip install transformers datasets peft bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

### Defining Model

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

base_model = "EleutherAI/pythia-1b"

model = AutoModelForCausalLM.from_pretrained(base_model, load_in_8bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [11]:
for name, module in model.named_modules():
    print(name)


gpt_neox
gpt_neox.embed_in
gpt_neox.emb_dropout
gpt_neox.layers
gpt_neox.layers.0
gpt_neox.layers.0.input_layernorm
gpt_neox.layers.0.post_attention_layernorm
gpt_neox.layers.0.post_attention_dropout
gpt_neox.layers.0.post_mlp_dropout
gpt_neox.layers.0.attention
gpt_neox.layers.0.attention.query_key_value
gpt_neox.layers.0.attention.dense
gpt_neox.layers.0.mlp
gpt_neox.layers.0.mlp.dense_h_to_4h
gpt_neox.layers.0.mlp.dense_4h_to_h
gpt_neox.layers.0.mlp.act
gpt_neox.layers.1
gpt_neox.layers.1.input_layernorm
gpt_neox.layers.1.post_attention_layernorm
gpt_neox.layers.1.post_attention_dropout
gpt_neox.layers.1.post_mlp_dropout
gpt_neox.layers.1.attention
gpt_neox.layers.1.attention.query_key_value
gpt_neox.layers.1.attention.dense
gpt_neox.layers.1.mlp
gpt_neox.layers.1.mlp.dense_h_to_4h
gpt_neox.layers.1.mlp.dense_4h_to_h
gpt_neox.layers.1.mlp.act
gpt_neox.layers.2
gpt_neox.layers.2.input_layernorm
gpt_neox.layers.2.post_attention_layernorm
gpt_neox.layers.2.post_attention_dropout
gpt_n

### PEFT Config

In [12]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value", "dense"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


In [13]:
model = get_peft_model(model, config)

### Define Dataset

In [14]:
toy_data = [
    {
        "prompt": "Translate to Portuguese: 'Hello!'",
        "completion": "Olá!"
    },
    {
        "prompt": "Translate to Portuguese: 'How are you?'",
        "completion": "Como você está?"
    },
    {
        "prompt": "Translate to Portuguese: 'Good morning.'",
        "completion": "Bom dia."
    },
    {
        "prompt": "Translate to Portuguese: 'Good night.'",
        "completion": "Boa noite."
    },
    {
        "prompt": "Translate to Portuguese: 'Thank you very much.'",
        "completion": "Muito obrigado."
    },
    {
        "prompt": "Translate to Portuguese: 'See you later!'",
        "completion": "Até logo!"
    },
    {
        "prompt": "Translate to Portuguese: 'What is your name?'",
        "completion": "Qual é o seu nome?"
    },
    {
        "prompt": "Translate to Portuguese: 'My name is John.'",
        "completion": "Meu nome é John."
    },
    {
        "prompt": "Translate to Portuguese: 'I love learning languages.'",
        "completion": "Eu adoro aprender idiomas."
    },
    {
        "prompt": "Translate to Portuguese: 'Where is the nearest restaurant?'",
        "completion": "Onde fica o restaurante mais próximo?"
    }
]

In [4]:
from datasets import load_dataset

ds = load_dataset("acon96/Home-Assistant-Requests", split='train')
print(ds)


Dataset({
    features: ['conversations'],
    num_rows: 33361
})


In [10]:
messages = ds['conversations']

def format_chat(example):
    #for m in example['conversations']:
     #   print(m)
    teste = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    print(teste)

format_chat(ds[0])


def format_chat_and_tokenize(example):


    #full_text =
    #tokenizer.

    return tokenizer(full_text, truncation=True, padding="max_length", max_length=128)



#ds.map(format_chat_and_tokenize)




#for m in messages:
#    print(m)

NameError: name 'tokenizer' is not defined

In [15]:
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling

dataset = Dataset.from_list(toy_data)

print(dataset)

def join_and_tokenize(example):
    full_text = example["prompt"] + " " + example["completion"]
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(join_and_tokenize)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 10
})


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### Define Trainer

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./pythia_lora_pt",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=50,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    save_steps=20,
    save_total_limit=2,
    label_names=["labels"],
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

In [17]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.201299031575521, metrics={'train_runtime': 5.962, 'train_samples_per_second': 5.032, 'train_steps_per_second': 0.503, 'total_flos': 20974047068160.0, 'train_loss': 3.201299031575521, 'epoch': 3.0})